In [ ]:
from pyrpipe import sra,mapping,assembly,qc,tools
#First get the srr accessions of the runs. For this one can use the python package pysradb or R package sradb
runsSmol=[]
maize_data=""
working_dir=""